<center><h1/><br>Graph Classification<br></h1></center>



The goal of this lab is to introduce machine learning techniques for graph classification. 
In order to perform graph classification, we will employ graph kernels, a powerful framework for graph comparison.
Kernels can be intuitively understood as functions measuring the similarity of pairs of objects. More formally, for a function $k(x,x')$ to be a kernel, it has to be (1) symmetric: $k(x,x') = k(x',x)$, and (2) positive semi-definite. If a function satisfies the above two conditions on a set $\mathcal{X}$, it is known that there exists a map $\phi : \mathcal{X} \to \mathcal{H}$ into a Hilbert space $\mathcal{H}$, such that $k(x,x') = \langle \phi(x), \phi(x') \rangle$ for all $(x, x') \in \mathcal{X}^2$ where $\langle\cdot, \cdot\rangle$ is the inner product in $\mathcal{H}$. Kernel functions thus compute the inner product between examples that are mapped in a higher-dimensional feature space. However, they do not necessarily explicitly compute the feature map $\phi$ for each example. One advantage of kernel methods is that they can operate on very general types of data such as images and graphs. Kernels defined on graphs are known as *graph kernels*. Most graph kernels decompose graphs into their substructures and then to measure their similarity, they count the number of common substructures. Graph kernels typically focus on some structural aspect of graphs such as random walks, shortest paths, subtrees, cycles, and graphlets.

We will first create a very simple graph classification dataset. We will use the [NexworkX](http://networkx.github.io/) library to create and manipulate graphs.
The dataset will contain two types of graphs: (1) cycle graphs, and (2) path graphs. A cycle graph $C_n$ is a graph on $n$ nodes containing a single cycle through all nodes, while a path graph $P_n$ is a tree with two nodes of degree 1, and all the remaining $n-2$ nodes of degree 2. Each graph is assigned a class label: label 0 if it is a cycle or label 1 if it is a path. The Figure below illustrates such a dataset consisting of three cycle graphs and three path graphs.

<img src="synthetic_graphs.png" width="500"/>
    
<u>Task</u>:
- Use the [`cycle_graph()`](https://networkx.github.io/documentation/stable/reference/generated/networkx.generators.classic.cycle_graph.html#networkx.generators.classic.cycle_graph) and [`path_graph()`](https://networkx.github.io/documentation/stable/reference/generated/networkx.generators.classic.path_graph.html#networkx.generators.classic.path_graph) functions of NetworkX to generate 100 cycle graphs and 100 path graphs of size $n=3,\ldots,102$, respectively. Store the 200 graphs in a list $Gs$ and their class labels in another list $y$.

In [ ]:
import networkx as nx
import numpy as np

Gs = list()
y = list()

for i in range(3,103):
    # write your code here 

We will next investigate if graph kernels can distinguish cycle graphs from path graphs. To this end, we will make use of the shortest path kernel, a kernel that compares shortest path lengths in two graps. Before computing the kernel, it is necessary to split the dataset into a training and a test set. We can use the [`train_test_split()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function of scikit-learn as follows:
    
    from sklearn.model_selection import train_test_split

    G_train, G_test, y_train, y_test = train_test_split(Gs, y, test_size=0.1)
    
<u>Task</u>:
- Split the dataset into a training and a test set.

In [ ]:
from sklearn.model_selection import train_test_split

# write your code here

The shortest path kernel compares the length of shortest paths of two graphs. More specifically, given two graphs $G=(V,E)$ and $G'=(V',E')$, the shortest path kernel is defined as:

$$
k(G,G') = \sum_{(v_1,v_2) \in V \times V} \sum_{(v'_1,v'_2) \in V' \times V'} k_{length}(sp_{v_1,v_2}, sp_{v'_1,v'_2})
$$

where $k_{length}$ is a kernel on shortest path lengths, and $sp_{v_i,v_j}$ is the length of the shortest path between vertices $v_i$ and $v_j$. We will use the following kernel for comparing shortest path lengths:

$$
k_{length}(sp_{v_1,v_2}, sp_{v'_1,v'_2}) = \left\{
            \begin{array}{lr}
                1 & \text{if }sp_{v_1,v_2} = sp_{v'_1,v'_2},\\
                0 & \text{otherwise}
            \end{array}
            \right.
$$

Therefore, $k_{length}(sp_{v_1,v_2}, sp_{v'_1,v'_2})$ is equal to 1 if $sp_{v_1,v_2}$ and $sp_{v'_1,v'_2}$ are equal to each other, and $0$ otherwise.

Below you are given a function that takes as input two sets of graphs (of sizes $N_1$ and $N_2$), and computes the kernel matrix $K \in \mathbb{R}^{N_1\times N_2}$ which stores the kernel values between the graphs of the first set and those of the second set.

In [ ]:
def sp_kernel(Gs1, Gs2):
    N1 = len(Gs1)
    N2 = len(Gs2)
    
    all_paths = dict()
    sp_counts = dict()
    
    for i,G in enumerate(Gs1):
        sp_lengths = dict(nx.shortest_path_length(G))
        sp_counts[i] = dict()
        nodes = G.nodes()
        for v1 in nodes:
            for v2 in nodes:
                if v2 in sp_lengths[v1]:
                    length = sp_lengths[v1][v2]
                    if length in sp_counts[i]:
                        sp_counts[i][length] += 1
                    else:
                        sp_counts[i][length] = 1

                    if length not in all_paths:
                        all_paths[length] = len(all_paths)
                        
    for i,G in enumerate(Gs2):
        sp_lengths = dict(nx.shortest_path_length(G))
        sp_counts[N1+i] = dict()
        nodes = G.nodes()
        for v1 in nodes:
            for v2 in nodes:
                if v2 in sp_lengths[v1]:
                    length = sp_lengths[v1][v2]
                    if length in sp_counts[N1+i]:
                        sp_counts[N1+i][length] += 1
                    else:
                        sp_counts[N1+i][length] = 1

                    if length not in all_paths:
                        all_paths[length] = len(all_paths)

    phi1 = np.zeros((N1, len(all_paths)))
    for i in range(N1):
        for length in sp_counts[i]:
            phi1[i,all_paths[length]] = sp_counts[i][length]
            
    phi2 = np.zeros((N2, len(all_paths)))
    for i in range(N2):
        for length in sp_counts[N1+i]:
            phi2[i,all_paths[length]] = sp_counts[N1+i][length]

    K = np.dot(phi1,phi2.T)

    return K

We are interested in generating two matrices. A symmetric matrix $\mathbf{K}_{train}$ which contains the kernel values for all pairs of training graphs, and a second matrix $\mathbf{K}_{test}$ which stores the kernel values between the graphs of the test set and those of the training set. We can obtain these two matrices very easily using the function defined above. After generating the two kernel matrices, we can use the SVM classifier to perform graph classification.

<u>Tasks</u>:
- Use the shortest path kernel to compute the $\mathbf{K}_{train}$ and $\mathbf{K}_{test}$ matrices.
- Train an [SVM classifier](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) and use it to make predictions. Note that we have already pre-computed the kernel matrices (set parameter kernel equal to 'precomputed').  

In [ ]:
from sklearn.svm import SVC

# write your code here

Finally, we will evaluate the shortest path kernel. More specifically, we will compute its classification accuracy.

<u>Tasks</u>:
- Use the [`accuracy_score()`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html) function of scikit-learn to compute the classification accuracy of the shortest path kernel. 

In [ ]:
from sklearn.metrics import accuracy_score

# your code here